In [1]:
import os

import cmcrameri.cm as cmc
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
import scipy.stats as stats



In [2]:
def myround(x, base=5):
    return base * round(x/base)

In [3]:
combined = pd.read_csv('/home/earthquakes1/homes/Rebecca/phd/stf/data/combined.csv')

In [4]:
def get_stf(scardec_name, wanted_type = 'fctopt'):
    db = combined[combined['scardec_name']==scardec_name]

    time = []
    momentrate = []

    event = os.listdir(f'/home/earthquakes1/homes/Rebecca/phd/stf/data/scardec/{scardec_name}')
    starts = [n for n, l in enumerate(event) if l.startswith(wanted_type)]
    with open(f'/home/earthquakes1/homes/Rebecca/phd/stf/data/scardec/{scardec_name}/{event[starts[0]]}') as f:
        lines = f.read().splitlines()

    lines = lines[2:]
    for line in lines:
        split = line.split(' ')
        split = [s for s in split if s not in ['', ' ', '\n']]
        time.append(float(split[0]))
        momentrate.append(float(split[1]))

    momentrate = np.array(momentrate)
    return momentrate, time, db

In [ ]:
# looks for time value of root
def f3(end_time, total_moment, time_opt, momentrate_opt, start, points_before_zero, proportion = 0.1):
    dx = time_opt[1]-time_opt[0]
    end_window = (end_time/dx)+points_before_zero
    end = int(np.floor(end_window))
    if start == end:
        end += 1
    short = scipy.integrate.simpson(momentrate_opt[start:end], dx = dx)
    return short-(total_moment*proportion)


In [6]:
def plot_scatter_figures(proportion, root_times, moments, durations):
    root_times = np.array(root_times)
    moments = np.array(moments)
    durations = np.array(durations)

    plt.scatter(root_times, durations, c = np.log10(moments), cmap = cmc.batlow, alpha = 0.5)
    plt.ylabel('Duration (s)')
    plt.xlabel(f'time to release {proportion*100}% of moment (s)')
    plt.colorbar(label = 'log10(moment)')
    plt.savefig(f'/home/earthquakes1/homes/Rebecca/phd/stf/figures/moment_intervals/time_for_{proportion*100}_percent_moment_against_duration.png')
    plt.close()

    plt.scatter(root_times, np.log10(moments), c = durations, cmap = cmc.batlow, alpha = 0.5)
    plt.ylabel('log10(moment)')
    plt.xlabel(f'time to release {proportion*100}% of moment (s)')
    plt.colorbar(label = 'Duration (s)')
    plt.savefig(f'/home/earthquakes1/homes/Rebecca/phd/stf/figures/moment_intervals/time_for_{proportion*100}_percent_moment_against_moment.png')
    plt.close()

    plt.scatter(root_times/durations, np.log10(moments), c = durations, cmap = cmc.batlow, alpha = 0.5)
    plt.ylabel('log10(moment)')
    plt.xlabel(f'proportion of duration to release {proportion*100}% of moment')
    plt.colorbar()
    plt.xlim(0, 1)
    plt.savefig(f'/home/earthquakes1/homes/Rebecca/phd/stf/figures/moment_intervals/fraction_of_duration_for_{proportion*100}_percent_moment_against_moment.png')
    plt.close()

In [7]:
def plot_hist_figures(proportion, root_times, durations):
    root_times = np.array(root_times)
    durations = np.array(durations)

    plt.hist(root_times/durations, bins = 100)

    plt.ylabel('Frequency')
    plt.xlabel(f'Proportion of duration to release {proportion*100}% of moment')
    plt.xlim(0, 1)
    plt.show()
    #plt.savefig(f'/home/earthquakes1/homes/Rebecca/phd/stf/figures/moment_intervals/histogram_fraction_of_duration_for_{proportion*100}_percent_moment.png')
    #plt.close()

In [8]:
proportions_list = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

all_durations = []
all_root_times = []
all_moments = []
all_relative_root_times = []

df = pd.DataFrame(columns = ['proportion', 'skew', 'skew statistic', 'skew pvalue',  'kurtosis', 'kurtosis statistic', 'kurtosis pvalue'])

for proportion in proportions_list:
    print(proportion)
    durations = []
    root_times = []
    relative_root_times = []

    diff = []
    moments = []

    for scardec_name in os.listdir('/home/earthquakes1/homes/Rebecca/phd/stf/data/scardec'):
        #print(scardec_name)
        momentrate_opt, time_opt, db = get_stf(scardec_name, wanted_type='fctmoy')

        not_zero = np.where(momentrate_opt > 0)[0]

        dx = time_opt[1]-time_opt[0]

        start = min(not_zero)
        end = max(not_zero)
        points_before_zero = abs(min(time_opt)/dx)

        duration = time_opt[end] - time_opt[start]
        durations.append(duration)

        start_time = time_opt[start]
        end_time = time_opt[end]

        total_moment = scipy.integrate.simpson(momentrate_opt[start:end], dx = time_opt[1]-time_opt[0])
        moments.append(total_moment)
        root, r = scipy.optimize.bisect(f3,
                                        start_time+dx,
                                        end_time,
                                        rtol = 1e-6,
                                        full_output = True,
                                        args = (total_moment,
                                                time_opt,
                                                momentrate_opt,
                                                start,
                                                points_before_zero,
                                                proportion,))
        root_idx = np.floor(root/dx)
        root_time = root_idx*dx
        root_times.append(root_time)
        relative_root_times.append(root_time-start_time)

        if root_time-start_time > duration:
            print('root time greater than duration, proportion:', proportion)
            print(scardec_name)

    root_times = np.array(root_times)
    durations = np.array(durations)
    moments = np.log10(np.array(moments))
    relative_root_times = np.array(relative_root_times)

    rel_root_times = relative_root_times/durations

    skew = stats.skew(rel_root_times)
    kurtosis = stats.kurtosis(rel_root_times)
    kurtosis_test = stats.kurtosistest(rel_root_times)
    skew_test = stats.skewtest(rel_root_times)

    new_data = {'proportion': [proportion], 'skew': [skew], 'skew statistic': [skew_test[0]], 'skew pvalue': [skew_test[1]], 'kurtosis': [kurtosis], 'kurtosis statistic': [kurtosis_test[0]], 'kurtosis pvalue': [kurtosis_test[1]]}
    df = pd.concat([df, pd.DataFrame(new_data)])


    #plt.savefig(f'/home/earthquakes1/homes/Rebecca/phd/stf/figures/moment_intervals/stacked_histogram_fraction_of_duration_for_{proportion*100}_percent_moment.png')

0.1
0.2
0.3
0.4
0.5
0.6
0.7
0.8
0.9


Skew: For normally distributed data, the skewness should be about zero. For unimodal continuous distributions, a skewness value greater than zero means that there is more weight in the right tail of the distribution. 

Skew statistic: The computed z-score for this test.

Skew p-value: The test is performed by comparing the observed value of the statistic against the null distribution: the distribution of statistic values derived under the null hypothesis that the weights were drawn from a normal distribution.




In [9]:
df

,proportion,skew,skew statistic,skew pvalue,kurtosis,kurtosis statistic,kurtosis pvalue
0,0.1,0.778094,18.111504,2.585860e-73,0.799637,7.657245,1.899640e-14
0,0.2,0.615459,14.871510,5.046228e-50,0.331405,3.793640,1.484546e-04
0,0.3,0.544585,13.359025,1.049297e-40,0.179423,2.215365,2.673506e-02
0,0.4,0.486173,12.066195,1.593330e-33,0.016994,0.277723,7.812248e-01
0,0.5,0.427478,10.725740,7.706776e-27,-0.076037,-0.977158,3.284909e-01
0,0.6,0.334568,8.523176,1.552363e-17,-0.141646,-1.938619,5.254779e-02
0,0.7,0.173549,4.505557,6.619900e-06,-0.272646,-4.086239,4.384214e-05
0,0.8,0.007333,0.191786,8.479102e-01,-0.457771,-7.804406,5.978233e-15
0,0.9,-0.167678,-4.355228,1.329282e-05,-0.486276,-8.469531,2.463822e-17


In [10]:
print(df.to_latex(index=False,
                  formatters={"name": str.upper},
                  float_format="{:.3f}".format,
))

\begin{tabular}{rrrrrrr}
\toprule
proportion & skew & skew statistic & skew pvalue & kurtosis & kurtosis statistic & kurtosis pvalue \\
\midrule
0.100 & 0.778 & 18.112 & 0.000 & 0.800 & 7.657 & 0.000 \\
0.200 & 0.615 & 14.872 & 0.000 & 0.331 & 3.794 & 0.000 \\
0.300 & 0.545 & 13.359 & 0.000 & 0.179 & 2.215 & 0.027 \\
0.400 & 0.486 & 12.066 & 0.000 & 0.017 & 0.278 & 0.781 \\
0.500 & 0.427 & 10.726 & 0.000 & -0.076 & -0.977 & 0.328 \\
0.600 & 0.335 & 8.523 & 0.000 & -0.142 & -1.939 & 0.053 \\
0.700 & 0.174 & 4.506 & 0.000 & -0.273 & -4.086 & 0.000 \\
0.800 & 0.007 & 0.192 & 0.848 & -0.458 & -7.804 & 0.000 \\
0.900 & -0.168 & -4.355 & 0.000 & -0.486 & -8.470 & 0.000 \\
\bottomrule
\end{tabular}



In [11]:
stats.skewtest(rel_root_times)

SkewtestResult(statistic=-4.3552284436687945, pvalue=1.3292821597683676e-05)